In [1]:
!guardrails hub install hub://guardrails/lowercase
!guardrails hub install hub://guardrails/two_words
!guardrails hub install hub://guardrails/one_line

Installing hub://guardrails/lowercase...
[ ===] Fetching manifestst
[=   ] Downloading dependencies  Running command git clone --filter=blob:none --quiet https://github.com/guardrails-ai/lowercase.git /private/var/folders/c8/jqt82fpx785dpwpp36ljkgm40000gn/T/pip-req-build-iryeqinr
[ ===] Downloading dependenciesWARNING: Target directory /Users/zaydsimjee/workspace/guardrails/docs/examples/.venv/lib/python3.10/site-packages/guardrails/hub/guardrails/lowercase/validator already exists. Specify --upgrade to force replacement.
[=   ] Downloading dependencies
[    ] Running post-install setup
✅Successfully installed guardrails/lowercase!


Import validator:
from guardrails.hub import LowerCase

Get more info:
https://hub.guardrailsai.com/validator/guardrails/lowercase

Installing hub://guardrails/two_words...
[  ==] Fetching manifestst
[=   ] Downloading dependencies  Running command git clone --filter=blob:none --quiet https://github.com/guardrails-ai/two_words.git /private/var/folders/c8/j

# Using Guardrails with Chat Models

!!! note
    To download this example as a Jupyter notebook, click [here](https://github.com/guardrails-ai/guardrails/blob/main/docs/examples/extracting_entities.ipynb).

In this example, we will set up Guardrails with a chat model.

## Objective

We retry the [entity extraction example](./extracting_entities.ipynb) using a chat model.

## Step 0: Download PDF and load it as string

To get started, download the document from [here](https://github.com/guardrails-ai/guardrails/blob/main/docs/examples/data/chase_card_agreement.pdf) and save it in `data/chase_card_agreement.pdf`.

Guardrails has some built-in functions to help with common tasks. Here, we will use the `read_pdf` function to load the PDF as a string.

In [2]:
import guardrails as gd
from rich import print

content = gd.docs_utils.read_pdf("data/chase_card_agreement.pdf")

print(f"Chase Credit Card Document:\n\n{content[:275]}\n...")

/Users/zaydsimjee/workspace/guardrails/docs/examples/.venv/lib/python3.10/site-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


Chase Credit Card Document:

2/25/23, 7:59 PM about:blank
about:blank 1/4
PRICING INFORMATION
INTEREST RATES AND INTEREST CHARGES
Purchase Annual
Percentage Rate (APR) 0% Intro APR for the first 18 months that your Account is open.
After that, 19.49%. This APR will vary with the market based on the Prim
...

## Step 1: Create the RAIL Spec with `<instructions>` tags

In order to use Guardrails with a chat model, we need to add `<instructions>` tags to the RAIL spec. These tags will be used to generate the system message for the chat model.

Ordinarily, everything that is contained in the `<prompt>` tag will be split across `<prompt>` and `<instructions>` tags. Here's an example illustrating the differences.


=== "RAIL Spec with instruction tag"

    ```xml
    <instructions>
    You are a helpful assistant only capable of communicating with valid JSON, and no other text.

    ${gr.json_suffix_prompt_examples}
    </instructions>


    <prompt>
    Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 
    `null`.

    ${document}

    Extract information from this document and return a JSON that follows the correct schema.

    ${gr.xml_prefix_prompt}

    ${output_schema}
    </prompt>
    ```

=== "Pydantic with instructions"
    ```py
    instructions = """You are a helpful assistant only capable of communicating with valid JSON, and no other text.

    ${gr.json_suffix_prompt_examples}"""

    prompt = """Given the following document, answer the following questions. If the answer doesn't exist in the document, enter `null`.

    ${document}

    Extract information from this document and return a JSON that follows the correct schema.

    ${gr.xml_prefix_prompt}

    ${output_schema}"""
    ```

=== "RAIL Spec without instruction tag"
    
    ```xml
    <prompt>
    Given the following document, answer the following questions. If the answer doesn't exist in the document, enter `null`.

    ${document}

    ${gr.xml_prefix_prompt}

    ${output_schema}

    ${gr.json_suffix_prompt_v2_wo_none}
    </prompt>
    ```

=== "Pydantic without instructions"
    ```py
    prompt = """Given the following document, answer the following questions. If the answer doesn't exist in the document, enter `null`.

    ${document}

    ${gr.xml_prefix_prompt}

    ${output_schema}

    ${gr.json_suffix_prompt_v2_wo_none}"""
    ```

After materialization, the two variations of the RAIL specification will look like this when passed to the LLM (regarless if they were initialized from xml or a Pydantic model):

=== "RAIL Spec with instruction tag"

    ```xml
    <instructions>
    You are a helpful assistant only capable of communicating with valid JSON, and no other text.

    ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and specific types. Be correct and concise. If you are unsure anywhere, enter `null`.

    Here are examples of simple (XML, JSON) pairs that show the expected behavior:
    - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
    - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
    - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`
    </instructions>


    <prompt>
    Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 
    `null`.

    ${document}

    Extract information from this document and return a JSON that follows the correct schema.

    Given below is XML that describes the information to extract from this document and the tags to extract it into.

    ${output_schema}
    </prompt>
    ```

=== "RAIL Spec without instruction tag"

    ```xml
    <prompt>
    Given the following document, answer the following questions. If the answer doesn't exist in the document, enter `null`.

    ${document}

    Given below is XML that describes the information to extract from this document and the tags to extract it into.

    ${output_schema}

    ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and specific types. Be correct and concise. If you are unsure anywhere, enter `null`.

    Here are examples of simple (XML, JSON) pairs that show the expected behavior:
    - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
    - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
    - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`
    </prompt>
    ```

Here's the final RAIL spec as XML:

In [3]:
rail_str = """
<rail version="0.1">

<output>

    <list name="fees" description="What fees and charges are associated with my account?">
        <object>
            <integer name="index" format="1-indexed" />
            <string name="name" format="lower-case; two-words" on-fail-lower-case="fix" on-fail-two-words="reask" />
            <string name="explanation" format="one-line" />
            <float name="value" format="percentage"/>
        </object>
    </list>
    <object name="interest_rates" required="false" description="What are the interest rates offered by the bank on savings and checking accounts, loans, and credit products?" />
</output>


<instructions>
You are a helpful assistant only capable of communicating with valid JSON, and no other text.

${gr.json_suffix_prompt_examples}
</instructions>


<prompt>
Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 
`null`.

${document}

Extract information from this document and return a JSON that follows the correct schema.

${gr.xml_prefix_prompt}

${output_schema}
</prompt>

</rail>
"""

Or using a Pydantic model for the output schema:

In [4]:
from guardrails.hub import LowerCase, TwoWords, OneLine
from pydantic import BaseModel, Field
from typing import List, Optional

instructions = """You are a helpful assistant only capable of communicating with valid JSON, and no other text.

${gr.json_suffix_prompt_examples}"""

prompt = """Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 
`null`.

${document}

Extract information from this document and return a JSON that follows the correct schema.

${gr.xml_prefix_prompt}

${output_schema}"""

class Fee(BaseModel):
    index: int = Field(validators=[("1-indexed", "noop")])
    name: str = Field(validators=[LowerCase(on_fail="fix"), TwoWords(on_fail="reask")])
    explanation: str = Field(validators=[OneLine()])
    value: float = Field(validators=[("percentage", "noop")])

class CreditCardAgreement(BaseModel):
    fees: List[Fee] = Field(description="What fees and charges are associated with my account?")
    interest_rates: Optional[dict] = Field(description="What are the interest rates offered by the bank on savings and checking accounts, loans, and credit products?")

/Users/zaydsimjee/workspace/guardrails/docs/examples/.venv/lib/python3.10/site-packages/guardrails/validators/__init__.py:51: FutureWarning: 
    Importing validators from `guardrails.validators` is deprecated.
    All validators are now available in the Guardrails Hub. Please install
    and import them from the hub instead. All validators will be
    removed from this module in the next major release.

    Install with: `guardrails hub install hub://<namespace>/<validator_name>`
    Import as: from guardrails.hub import `ValidatorName`
    
  warn(


## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the output of the LLM. This object:

1. Enforces the quality criteria specified in the RAIL spec.
2. Takes corrective action when the quality criteria are not met.
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.

Creating the guard from XML:

In [5]:
guard = gd.Guard.from_rail_string(rail_str)

/Users/zaydsimjee/workspace/guardrails/docs/examples/.venv/lib/python3.10/site-packages/guardrails/validatorsattr.py:307: UserWarning: Validator 1-indexed is not installed!
  warnings.warn(f"Validator {validator_name} is not installed!")
/Users/zaydsimjee/workspace/guardrails/docs/examples/.venv/lib/python3.10/site-packages/guardrails/validator_base.py:410: FutureWarning: Accessing `LowerCase` using
`from guardrails.validators import LowerCase` is deprecated and
support will be removed after version 0.5.x. Please switch to the Guardrails Hub syntax:
`from guardrails.hub import LowerCase` for future updates and support.
For additional details, please visit: https://hub.guardrailsai.com/validator/guardrails/lowercase.

  warn(
/Users/zaydsimjee/workspace/guardrails/docs/examples/.venv/lib/python3.10/site-packages/guardrails/validator_base.py:410: FutureWarning: Accessing `TwoWords` using
`from guardrails.validators import TwoWords` is deprecated and
support will be removed after version 

Or from the Pydantic model:

In [6]:
guard = gd.Guard.from_pydantic(output_class=CreditCardAgreement, instructions=instructions, prompt=prompt)

As we can see, a few formatters weren't supported. These formatters won't be enforced in the output, but this information can still be used to generate a prompt.

We see the prompt that will be sent to the LLM. The `{document}` is substituted with the user provided value at runtime.

In [7]:
print(guard.base_prompt)

/var/folders/c8/jqt82fpx785dpwpp36ljkgm40000gn/T/ipykernel_87930/3983563700.py:1: DeprecationWarning: 'Guard.base_prompt' is deprecated and will be removed in versions 0.5.x and beyond. Use 'Guard.history.last.prompt' instead.
  print(guard.base_prompt)


Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 
`null`.

${document}

Extract information from this document and return a JSON that follows the correct schema.


Given below is XML that describes the information to extract from this document and the tags to extract it into.


<output>
    <list name="fees" description="What fees and charges are associated with my account?">
        <object>
            <integer name="index" format="1-indexed"/>
            <string name="name" format="guardrails/lowercase; guardrails/two_words"/>
            <string name="explanation" format="guardrails/one_line"/>
            <float name="value" format="percentage"/>
        </object>
    </list>
    <object name="interest_rates" description="What are the interest rates offered by the bank on savings and 
checking accounts, loans, and credit products?" required="false"/>
</output>

Here's the formatted instructions sent as the system message to the LLM.

In [8]:
import openai

raw_llm_response, validated_response, *rest = guard(
    openai.chat.completions.create,
    prompt_params={"document": content[:6000]},
    max_tokens=2048,
    temperature=0,
)

The `guard` wrapper returns the raw_llm_respose (which is a simple string), and the validated and corrected output (which is a dictionary).

We can see that the output is a dictionary with the correct schema and types.

In [9]:
print(validated_response)

None

In [10]:
guard.history.last.tree

Logs
├── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
│   │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
│   │ │ Given the following document, answer the following questions. If the answer doesn't exist in the        │ │
│   │ │ document, enter                                                                                         │ │
│   │ │ `null`.                                                                                                 │ │
│   │ │                                                                                                         │ │
│   │ │ 2/25/23, 7:59 PM about:blank                                                                            │ │
│   │ │ about:blank 1/4                                                                                         │ │
│   │ │ PRICING INFORMATION                                                                                     │ │
│   │ │ INTEREST RATES AND INTEREST CHARGES                                                                     │ │
│   │ │ Purchase Annual                                                                                         │ │
│   │ │ Percentage Rate (APR) 0% Intro APR for the first 18 months that your Account is open.                   │ │
│   │ │ After that, 19.49%. This APR will vary with the market based on the Prime                               │ │
│   │ │ Rate.                                                                                                   │ │
│   │ │ a                                                                                                       │ │
│   │ │ My Chase Loan                                                                                           │ │
│   │ │ SM APR 19.49%. This APR will vary with the market based on the Prime Rate.                              │ │
│   │ │ a                                                                                                       │ │
│   │ │ Promotional offers with fixed APRs and varying durations may be available from                          │ │
│   │ │ time to time on some accounts.                                                                          │ │
│   │ │ Balance Transfer APR 0% Intro APR for the first 18 months that your Account is open.                    │ │
│   │ │ After that, 19.49%. This APR will vary with the market based on the Prime                               │ │
│   │ │ Rate.                                                                                                   │ │
│   │ │ a                                                                                                       │ │
│   │ │ Cash Advance APR 29.49%. This APR will vary with the market based on the Prime Rate.                    │ │
│   │ │ b                                                                                                       │ │
│   │ │ Penalty APR and When                                                                                    │ │
│   │ │ It Applies                                                                                              │ │
│   │ │ Up to 29.99%. This APR will vary with the market based on the Prime Rate.                               │ │
│   │ │ c                                                                                                       │ │
│   │ │ We may apply the Penalty APR to your account if you:                                                    │ │
│   │ │ fail to make a Minimum Payment by the date and time that it is due; or                                  │ │
│   │ │ make a payment to us that is returned unpaid.                                                           │ │
│   │ │ How Long Will the Penalty APR Apply?: If we apply the Penalty APR for                                   │ │
│   │ │ either of these reasons, the Penalty APR co